#setup
- connect to google drive in this step

In [ ]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

In [ ]:
def get_data(individual, bodypart, h5_file):
  mouse_data = h5_file.xs(individual,level='individuals',axis=1)
  out_data = mouse_data.xs(bodypart,level='bodyparts',axis=1)
  out_data.columns = out_data.columns.droplevel("scorer")
  output = out_data.copy()
  return output

In [ ]:
def within_area(area_vector, input_coor):
  area_startx = area_vector[0]
  area_starty = area_vector[1]
  area_distx = area_vector[2]
  area_disty = area_vector[3]
  x = input_coor["x"].iloc[0]
  y = input_coor["y"].iloc[0]
  if (area_startx <= x <= (area_startx+area_distx)) and (area_starty <= y <= (area_starty+area_disty)):
    result = 1
  else:
    result = 0
  return result

In [ ]:
def euclid_dist(point1_coor, point2_coor):
  point1 = np.array((point1_coor["x"].iloc[0], point1_coor["y"].iloc[0]))
  point2 = np.array((point2_coor["x"].iloc[0], point2_coor["y"].iloc[0]))
  output_dist = np.linalg.norm(point1 - point2)
  return output_dist

#on_male_side and putative mounts by disappearing female

Note: dlc origin is in the top left; x goes to the right, y goes down

In [ ]:
# for video in glob.glob(project_path + "/videos/" + "*.avi"):
for video in [project_path + "/videos/PZ82_2.avi"]:
  video_name = os.path.splitext(os.path.basename(video))[0]

  # area_vec = [area_startx, area_starty, area_distx, area_disty]
  female_side_mat = scipy.io.loadmat(project_path + "/videos/" + video_name + "_female_side.mat")
  female_side_vec = female_side_mat['croprect'][0]
  male_side_mat = scipy.io.loadmat(project_path + "/videos/" + video_name + "_male_side.mat")
  male_side_vec = male_side_mat['croprect'][0]

  h5_file = pd.read_hdf(project_path +'/videos/'+ video_name + 'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered.h5')
  [nframes, ncols] = h5_file.shape

  try:
    mus1_snout = get_data('mus1', 'snout', h5_file)
    mus1_shoulder = get_data('mus1', 'shoulder', h5_file)
    mus1_spine1 = get_data('mus1', 'spine1', h5_file)
    mus1_spine2 = get_data('mus1', 'spine2', h5_file)
    mus1_spine3 = get_data('mus1', 'spine3', h5_file)
    mus1_spine4 = get_data('mus1', 'spine4', h5_file)
    mus1_tailbase = get_data('mus1', 'tailbase', h5_file)

    mus2_snout = get_data('mus2', 'snout', h5_file)
    mus2_shoulder = get_data('mus2', 'shoulder', h5_file)
    mus2_spine1 = get_data('mus2', 'spine1', h5_file)
    mus2_spine2 = get_data('mus2', 'spine2', h5_file)
    mus2_spine3 = get_data('mus2', 'spine3', h5_file)
    mus2_spine4 = get_data('mus2', 'spine4', h5_file)
    mus2_tailbase = get_data('mus2', 'tailbase', h5_file)

    mus3_snout = get_data('mus3', 'snout', h5_file)
    mus3_shoulder = get_data('mus3', 'shoulder', h5_file)
    mus3_spine1 = get_data('mus3', 'spine1', h5_file)
    mus3_spine2 = get_data('mus3', 'spine2', h5_file)
    mus3_spine3 = get_data('mus3', 'spine3', h5_file)
    mus3_spine4 = get_data('mus3', 'spine4', h5_file)
    mus3_tailbase = get_data('mus3', 'tailbase', h5_file)
  except BaseException as err:
    try:
      mus1_snout = get_data('ind1', 'snout', h5_file)
      mus1_shoulder = get_data('ind1', 'shoulder', h5_file)
      mus1_spine1 = get_data('ind1', 'spine1', h5_file)
      mus1_spine2 = get_data('ind1', 'spine2', h5_file)
      mus1_spine3 = get_data('ind1', 'spine3', h5_file)
      mus1_spine4 = get_data('ind1', 'spine4', h5_file)
      mus1_tailbase = get_data('ind1', 'tailbase', h5_file)

      mus2_snout = get_data('ind2', 'snout', h5_file)
      mus2_shoulder = get_data('ind2', 'shoulder', h5_file)
      mus2_spine1 = get_data('ind2', 'spine1', h5_file)
      mus2_spine2 = get_data('ind2', 'spine2', h5_file)
      mus2_spine3 = get_data('ind2', 'spine3', h5_file)
      mus2_spine4 = get_data('ind2', 'spine4', h5_file)
      mus2_tailbase = get_data('ind2', 'tailbase', h5_file)

      mus3_snout = mus1_snout.copy()
      mus3_snout["x"].values[:] = 0
      mus3_snout["y"].values[:] = 0
      mus3_shoulder = mus3_snout.copy()
      mus3_spine1 = mus3_snout.copy()
      mus3_spine2 = mus3_snout.copy()
      mus3_spine3 = mus3_snout.copy()
      mus3_spine4 = mus3_snout.copy()
      mus3_tailbase = mus3_snout.copy()
    
    except BaseException as err:
      mus3_snout = mus1_snout.copy()
      mus3_snout["x"].values[:] = 0
      mus3_snout["y"].values[:] = 0
      mus3_shoulder = mus3_snout.copy()
      mus3_spine1 = mus3_snout.copy()
      mus3_spine2 = mus3_snout.copy()
      mus3_spine3 = mus3_snout.copy()
      mus3_spine4 = mus3_snout.copy()
      mus3_tailbase = mus3_snout.copy()


  behavior = pd.DataFrame(index=range(nframes),columns=['behavior'])
  behavior_events = pd.DataFrame(index=range(nframes - 1), columns=['events_index', 'minutes', 'seconds', 'behavior_events'])

  for i in range(nframes):
    event_name = 'male_side'

    if within_area(female_side_vec, mus1_snout.loc[[i]]) or within_area(female_side_vec, mus1_shoulder.loc[[i]]) or within_area(female_side_vec, mus1_spine1.loc[[i]]) \
    or within_area(female_side_vec, mus1_spine2.loc[[i]]) or within_area(female_side_vec, mus1_spine3.loc[[i]]) or within_area(female_side_vec, mus1_spine4.loc[[i]]) or within_area(female_side_vec, mus1_tailbase.loc[[i]]) \
    or within_area(female_side_vec, mus2_snout.loc[[i]]) or within_area(female_side_vec, mus2_shoulder.loc[[i]]) or within_area(female_side_vec, mus2_spine1.loc[[i]]) \
    or within_area(female_side_vec, mus2_spine2.loc[[i]]) or within_area(female_side_vec, mus2_spine3.loc[[i]]) or within_area(female_side_vec, mus2_spine4.loc[[i]]) or within_area(female_side_vec, mus2_tailbase.loc[[i]]) \
    or within_area(female_side_vec, mus3_snout.loc[[i]]) or within_area(female_side_vec, mus3_shoulder.loc[[i]]) or within_area(female_side_vec, mus3_spine1.loc[[i]]) \
    or within_area(female_side_vec, mus3_spine2.loc[[i]]) or within_area(female_side_vec, mus3_spine3.loc[[i]]) or within_area(female_side_vec, mus3_spine4.loc[[i]]) or within_area(female_side_vec, mus3_tailbase.loc[[i]]):
      event_name = 'female_side'

    else:
      if not (within_area(male_side_vec, mus1_spine1.loc[[i]]) or within_area(male_side_vec, mus1_spine2.loc[[i]]) or within_area(male_side_vec, mus1_spine3.loc[[i]]) or within_area(male_side_vec, mus1_spine4.loc[[i]]) or within_area(male_side_vec, mus1_tailbase.loc[[i]])) \
      and not (within_area(male_side_vec, mus2_spine1.loc[[i]]) or within_area(male_side_vec, mus2_spine2.loc[[i]]) or within_area(male_side_vec, mus2_spine3.loc[[i]]) or within_area(male_side_vec, mus2_spine4.loc[[i]]) or within_area(male_side_vec, mus2_tailbase.loc[[i]])) \
      and (within_area(male_side_vec, mus3_snout.loc[[i]]) and within_area(male_side_vec, mus3_tailbase.loc[[i]])):
        event_name = 'mount'
      elif not (within_area(male_side_vec, mus1_spine1.loc[[i]]) or within_area(male_side_vec, mus1_spine2.loc[[i]]) or within_area(male_side_vec, mus1_spine3.loc[[i]]) or within_area(male_side_vec, mus1_spine4.loc[[i]]) or within_area(male_side_vec, mus1_tailbase.loc[[i]])) \
      and (within_area(male_side_vec, mus2_snout.loc[[i]]) and within_area(male_side_vec, mus2_tailbase.loc[[i]])) \
      and not (within_area(male_side_vec, mus3_spine1.loc[[i]]) or within_area(male_side_vec, mus3_spine2.loc[[i]]) or within_area(male_side_vec, mus3_spine3.loc[[i]]) or within_area(male_side_vec, mus3_spine4.loc[[i]]) or within_area(male_side_vec, mus3_tailbase.loc[[i]])):
        event_name = 'mount'
      elif (within_area(male_side_vec, mus1_snout.loc[[i]]) and within_area(male_side_vec, mus1_tailbase.loc[[i]])) \
      and not (within_area(male_side_vec, mus2_spine1.loc[[i]]) or within_area(male_side_vec, mus2_spine2.loc[[i]]) or within_area(male_side_vec, mus2_spine3.loc[[i]]) or within_area(male_side_vec, mus2_spine4.loc[[i]]) or within_area(male_side_vec, mus2_tailbase.loc[[i]])) \
      and not (within_area(male_side_vec, mus3_spine1.loc[[i]]) or within_area(male_side_vec, mus3_spine2.loc[[i]]) or within_area(male_side_vec, mus3_spine3.loc[[i]]) or within_area(male_side_vec, mus3_spine4.loc[[i]]) or within_area(male_side_vec, mus3_tailbase.loc[[i]])):
        event_name = 'mount'

    behavior["behavior"][i] = event_name

  events_n = 0
  for i in range(1, nframes):
    if behavior["behavior"][i] != behavior["behavior"][i-1]:
      behavior_events["events_index"][events_n] = i
      behavior_events["minutes"][events_n] = i / (30*60)
      behavior_events["seconds"][events_n] = round(i / 30) % 60
      behavior_events["behavior_events"][events_n] = behavior["behavior"][i]
      events_n = events_n + 1
  print(events_n)
  behavior_events = behavior_events.dropna(how='all')

  total_events = behavior_events.shape[0]
  for events_n in range(1, total_events - 1):
    # false positive male side due to zaps
    before_event = behavior_events["behavior_events"][events_n - 1]
    after_event = behavior_events["behavior_events"][events_n + 1]
    start_index = behavior_events["events_index"][events_n]
    end_index = behavior_events["events_index"][events_n + 1]
    current_event = behavior_events["behavior_events"][events_n]

    if end_index - start_index < 10 and before_event == after_event and current_event == 'male_side' and before_event == 'female_side':
      behavior["behavior"][start_index:end_index] = 'female_side'
      behavior_events["behavior_events"][events_n] = float('nan')
      behavior_events["behavior_events"][events_n + 1] = float('nan')

  # clean up
  behavior_events = behavior_events.dropna()
  behavior_events = behavior_events.reset_index(drop=True)
  print(behavior_events.shape)

  total_events = behavior_events.shape[0]
  for events_n in range(1, total_events - 1):\

    # mounts between two female side events
    if behavior_events["behavior_events"][events_n] == 'mount':
      beforemount_event = behavior_events["behavior_events"][events_n - 1]
      aftermount_event = behavior_events["behavior_events"][events_n + 1]
      mount_start_index = behavior_events["events_index"][events_n]
      mount_end_index = behavior_events["events_index"][events_n + 1]

      # or 220919
      if beforemount_event == 'female_side' or aftermount_event == 'female_side':
        behavior["behavior"][mount_start_index:mount_end_index] = 'female_side'
        behavior_events["behavior_events"][events_n] = float('nan')
        if beforemount_event == 'female_side' and aftermount_event == 'female_side':
          behavior_events["behavior_events"][events_n + 1] = float('nan')
      elif mount_end_index - mount_start_index < 30:
        behavior["behavior"][mount_start_index:mount_end_index] = 'male_side'
        behavior_events["behavior_events"][events_n] = float('nan')
        behavior_events["behavior_events"][events_n + 1] = float('nan')

  # clean up
  behavior_events = behavior_events.dropna()
  behavior_events = behavior_events.reset_index(drop=True)

  with open(output_dir + video_name + '_behavior.pickle', 'wb') as f:
      pickle.dump(behavior, f)
  with open(output_dir + video_name + '_behavior_events.pickle', 'wb') as f:
      pickle.dump(behavior_events, f)

FileNotFoundError: ignored

Behavior categorization logic:
- if mouse detected on female side, -> definitely female_side

- if no mouse detected on female side, -> putative male_side
 - problem: when grooming, sometimes does not detect lone female on female side

- if no mouse detected on female side, only 1 mouse back detected on male side, and both adjacent events were female_side, -> likely undetected female_side

- if no mouse detected on female side, only 1 mouse back detected on male side, and at least 1 adjacent event was not female_side -> putative mount
 - problem: sometimes putative mounts are too quick to really be mounts

- if putative mount and lasts more than 1 second or 30 frames, -> likely mount


at this point:
- 'female_side' means mouse detected on female side
- 'male_side' means not 1 (so 0 or 2) mouse back detected on male side and no mice on female side
- 'mount' means 1 mouse back detected on male side and no mice on female side

#behavior video

In [ ]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14/'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

end_frame = 36000
for video in [project_path + "/videos/PZ81_1.avi"]:
  # for video in glob.glob(project_path + "/videos/" + "*.avi"):
  video_name = os.path.splitext(os.path.basename(video))[0]
  
  labeled_video = video_name+'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered_id_labeled.mp4'

  behavior_file = open(output_dir + video_name + '_behavior.pickle', 'rb')
  behavior = pickle.load(behavior_file)
  behavior_file.close()

  behavior_events_file = open(output_dir + video_name + '_behavior_events.pickle', 'rb')
  behavior_events = pickle.load(behavior_events_file)
  behavior_events_file.close()

  cap = cv2.VideoCapture(project_path + 'videos/' + labeled_video)

  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")

  while cap.isOpened():
    ret,frame = cap.read()
    print(frame.shape)
    frame_width = int(frame.shape[1])
    frame_height = int(frame.shape[0])
    break

  output_video = output_dir + video_name + '_behavior_video.mp4'

  out = cv2.VideoWriter(output_video,cv2.VideoWriter_fourcc(*'MP4V'), 30, (frame_width,frame_height))

  count = 0
  while cap.isOpened():
    ret,frame = cap.read()
    current_behavior = behavior["behavior"][count]
    text_color = (0,0,0)
    if current_behavior == 'male_side':
      text_color = (0, 0, 150)
    elif current_behavior == 'mount':
      text_color = (0, 0, 255)
    elif current_behavior == 'female_side':
      text_color = (50, 0 , 0)
    else:
      print(current_behavior)
      print(count)
    cv2.putText(frame, str(current_behavior), (25, 25), cv2.FONT_HERSHEY_DUPLEX, 1, text_color, 2)
    out.write(frame)

    count = count + 1
    if count > end_frame:
      break
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  cap.release()
  out.release()

(160, 266, 3)


#read previous behavior data pickles and edit behavior events

In [ ]:
video_name = 'PZ81_1'

behavior_file = open(output_dir + video_name + '_behavior.pickle', 'rb')
behavior = pickle.load(behavior_file)
behavior_file.close()

behavior_events_file = open(output_dir + video_name + '_behavior_events.pickle', 'rb')
behavior_events = pickle.load(behavior_events_file)
behavior_events_file.close()

print(video_name)
first_20_min = behavior.truncate(after=(36000 - 1)).copy()
male_side_seconds = (first_20_min[first_20_min["behavior"] == 'male_side']["behavior"].count() + first_20_min[first_20_min["behavior"] == 'mount']["behavior"].count()) / 30
male_side_percent = male_side_seconds / (36000 / 30)
mount_seconds = first_20_min[first_20_min["behavior"] == 'mount']["behavior"].count() / 30

first_20_min_events = behavior_events.copy()
male_side_entrances = first_20_min_events[first_20_min_events["behavior_events"] == 'male_side']["behavior_events"].count()
mount_events = first_20_min_events[first_20_min_events["behavior_events"] == 'mount']["behavior_events"].count()

print('male_side seconds: ', male_side_seconds)
print('male_side percent: ', male_side_percent)
print('mount seconds: ', mount_seconds)
print('mount events: ', mount_events)

PZ71_2
male_side seconds:  384.6333333333333
male_side percent:  0.32052777777777774
mount seconds:  27.666666666666668
mount events:  16


In [ ]:
print(first_20_min_events[first_20_min_events["behavior_events"] == 'mount'])

        behavior
300  female_side
301  female_side
302  female_side
303  female_side
304  female_side
..           ...
495  female_side
496  female_side
497  female_side
498  female_side
499  female_side

[200 rows x 1 columns]


In [ ]:
def change_behavior_event(behavior, behavior_events, event_index, new_event):
  behavior_events_i = behavior_events.loc[behavior_events['events_index'] == event_index]
  behavior_events["behavior_events"][behavior_events_i] = new_event
  start_index = behavior_events["events_index"][behavior_events_i]
  end_index = behavior_events["events_index"][behavior_events_i + 1]
  behavior["behavior"][start_index:end_index]

In [ ]:
event_index = 1
new_event = 'female_side'
change_behavior_event(behavior, behavior_events, new_event)

#mount forest features

In [ ]:
mouse1_feature_points = {}
mouse1_feature_points['mus1_snout'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_shoulder'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_spine1'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_spine2'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_spine3'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_spine4'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['mus1_tailbase'] = get_data('mus1', 'snout', h5_file)

mouse2_feature_points = {}
mouse2_feature_points['mus2_snout'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_shoulder'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_spine1'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_spine2'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_spine3'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_spine4'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['mus2_tailbase'] = get_data('mus2', 'snout', h5_file)

- 7 points on the male
- 7 points on the female
- male points to every female point: 7 * 7 = 49 distances
- angles between every 3 points?

In [ ]:
features = pd.DataFrame(index=range(nframes),columns=range(49))

the following code may take a while if there are a lot of frames

In [ ]:
col_num = 0
for mouse1_key, mouse1_point in mouse1_feature_points.items():
  for mouse2_key, mouse2_point in mouse2_feature_points.items():
    for i in range(10):
      row_dist = euclid_dist(mouse1_point.loc[[i]], mouse2_point.loc[[i]])
      features[col_num][i] = row_dist
    col_num = col_num + 1

In [ ]:
training_features = np.array([[7, 2], [3, 4]])
training_labels = np.array([[1, 0]])

In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(training_features, training_labels, test_size = 0.25, random_state = 42)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

(1000, 20) (1000,)


In [ ]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
# New random forest with only the two most important variables
rf_most_important = RandomForestRegressor(n_estimators= 1000, random_state=42)
# Extract the two most important features
important_indices = [feature_list.index('temp_1'), feature_list.index('average')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]
# Train the random forest
rf_most_important.fit(train_important, train_labels)
# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - test_labels)
# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

In [ ]:
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline
# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

##Features:
## 1) Cage side of female mouse at each frame: _on_male_side_
- Location of female mouse's snout
- Overlay of male side in the cage
- Check whether the x and y coordinates of female mouse's snout are within the male side
## 2) Male mounting at each frame: _mount_
- Location of male mouse's snout, shoulders, spine points, and tailbase
- Location of female mouse's spine points and tailbase
- Euclidean distance between all male points and all female point
## 3) Female investigation of the male at each frame: _sniffing_
- Location of female mouse's snout
- Location of male mouse's tailbase and spine points
- Euclidean distance between female mouse's snout and all male mouse points

###Features List:
## _on_male_side_
- female mouse's snout
- male side coordinate boundaries
- __True or False__

## _mount_
- euclid_dist between male snout, shoulder, spine1, spine2, spine3, spine4, tailbase
- & female spine1, spine2, spine3, spine4, tailbase
- __Random Forest?__